In [ ]:
from analysis.analysis import *
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
def heatmap(ax, F, bounds, xlabel=False, ylabel=False, cbar=False, cbar_label=False):
    xmin, xmax, vmin, vmax, nbins = bounds
    ax.set_xlabel(r"$x$ ($\mu$m)") if xlabel else ...
    ax.set_ylabel(r"$v$ ($\mu$m/hr)") if ylabel else ...
    _kwargs = {
        "extent": [xmin, xmax, vmin, vmax],
        "origin": "lower",
        "cmap": "jet",
        "interpolation": "bilinear",
    }
    im = ax.imshow(F, **_kwargs)
    if cbar:
        cbar = plt.colorbar(im, ax=ax)
        cbar.formatter.set_powerlimits((0, 0))
        cbar.formatter.set_useMathText(True)
    cbar.set_label(r"$F$ ($\mu$m/hr$^2$)") if cbar_label else ...
    ax.set_aspect("auto")


def streamplot(
    ax, F, bounds, title="", n_skip=1, zero_line=False, heatmap=False, **kwargs
):
    xmin, xmax, vmin, vmax, nbins = bounds
    ax.set_xlabel(r"$x$ ($\mu$m)")
    ax.set_ylabel(r"$v$ ($\mu$m/hr)")

    if heatmap:
        _kwargs = {
            "extent": [xmin, xmax, vmin, vmax],
            "origin": "lower",
            "cmap": "jet",
            "interpolation": "bilinear",
        }
        im = ax.imshow(F, **_kwargs)
        cbar = plt.colorbar(im, ax=ax)
        cbar.set_label(r"$F$ ($\mu$m/hr$^2$)")
        ax.set_aspect("auto")

    X, Y, init_pts = full_lattice(F, xmin, xmax, vmin, vmax, nbins)
    init_pts = init_pts[::n_skip]
    stream_kwargs = kwargs | {"start_points": init_pts}
    ax.streamplot(X, Y, Y, F, **stream_kwargs)
    ax.set_title(title)
    if zero_line:
        ax.hlines(0, *ax.get_xlim(), lw=1, color="black", zorder=-1)


def bistable_streamplot(
    ax,
    F,
    bounds,
    labels,
    end_pts,
    title="",
    n_skip=1,
    zero_line=False,
    heatmap=False,
    **kwargs
):
    xmin, xmax, vmin, vmax, nbins = bounds
    ax.set_xlabel(r"$x$ ($\mu$m)")
    ax.set_ylabel(r"$v$ ($\mu$m/hr)")

    if heatmap:
        _kwargs = {
            "extent": [xmin, xmax, vmin, vmax],
            "origin": "lower",
            "cmap": "jet",
            "interpolation": "bilinear",
        }
        im = ax.imshow(F, **_kwargs)
        cbar = plt.colorbar(im, ax=ax)
        cbar.set_label(r"$F$ ($\mu$m/hr$^2$)")
        ax.set_aspect("auto")

    X, Y, init_pts = full_lattice(F, xmin, xmax, vmin, vmax, nbins)
    init_pts_sub = init_pts[::n_skip]
    labels_sub = labels[::n_skip]
    for color in set(labels_sub):
        inits_ = init_pts_sub[np.where(np.array(labels_sub) == color)[0]]
        stream_kwargs = kwargs | {"color": color, "start_points": inits_}
        ax.streamplot(X, Y, Y, F, **stream_kwargs)
        ax.scatter(
            end_pts[:, 0], end_pts[:, 1], s=50, marker="*", color="black", zorder=10
        )
        img = lattice_to_image(init_pts, labels, bounds)
        septx = get_separatrices(
            img, levels=[0.5], origin="lower", extent=[xmin, xmax, vmin, vmax]
        )
        ax.plot(septx[:, 0], septx[:, 1], lw=2, color="black")

    ax.set_title(title)
    if zero_line:
        ax.hlines(0, *ax.get_xlim(), lw=1, color="black", zorder=-1)


def dynamicsplot(ax, F, bounds):
    X, Y, init_pts = full_lattice(F, *bounds)

    # raw
    v, f = evaluate_F_v_x0(Y, F, bounds[-1])
    lim = np.fabs([v.min(), v.max()]).min()
    v_interp = np.linspace(-lim, lim, 30)
    f_interp = np.interp(v_interp, v, f)

    ax.plot(v_interp, f_interp, "-o", lw=2, color="black")
    ax.grid(which="major")
    ax.set_xlabel(r"$v$ ($\mu$m/hr)")
    ax.set_ylabel(r"$F(x\rightarrow 0)$ ($\mu$m/hr$^2$)")

In [ ]:
def process_gid(gid):
    import pickle

    data = pickle.load(open(f"maps/map_{gid}.pkl", "rb"))
    if len(data["labels"]) > 0:
        return data["F"], data["bounds"], data["labels"], data["end_pts"]
    return data["F"], data["bounds"]

In [ ]:
kwargs = {
    "density": 10,
    "linewidth": 0.5,
    "arrowsize": 1.0,
    "integration_direction": "forward",
    "broken_streamlines": False,
    "color": "cornflowerblue",
}

---

### Fig 1

In [ ]:
nrows, ncols = 2, 2
fig, axs = plt.subplots(nrows, ncols, figsize=(3.5 * ncols, 3 * nrows))

heatmap(axs[0, 0], *process_gid(10), xlabel=True, ylabel=True)
heatmap(axs[0, 1], *process_gid(26), xlabel=True, cbar=True, cbar_label=True)

streamplot(
    axs[0, 0], *process_gid(10), n_skip=6, zero_line=True, **kwargs | {"color": "black"}
)
streamplot(
    axs[0, 1], *process_gid(26), n_skip=6, zero_line=True, **kwargs | {"color": "black"}
)
axs[0, 1].set_ylabel("")

dynamicsplot(axs[1, 0], *process_gid(10))
dynamicsplot(axs[1, 1], *process_gid(26))
axs[1, 1].set_ylabel("")
plt.tight_layout()
fig.savefig("paper_plots/lc_stationary_defaults.pdf")

### Fig. 2

In [ ]:
nrows, ncols = 1, 3
fig, axs = plt.subplots(nrows, ncols, figsize=(2.5 * ncols, 3.5 * nrows), sharey=True)
nskip = 7

bistable_streamplot(
    axs[0],
    *process_gid(2),
    title=r"$R_0=15\mu\mathrm{m}$",
    n_skip=nskip,
    zero_line=True,
    **kwargs
)
streamplot(
    axs[1],
    *process_gid(10),
    title=r"$R_0=18\mu\mathrm{m}$",
    n_skip=nskip,
    zero_line=True,
    **kwargs
)
streamplot(
    axs[2],
    *process_gid(122),
    title=r"$R_0=21\mu\mathrm{m}$",
    n_skip=nskip,
    zero_line=True,
    **kwargs
)
[axs[k].set_ylabel("") for k in range(1, 3)]

plt.tight_layout()
fig.savefig("paper_plots/varying_cell_size_streams.pdf")

### Fig 3

In [ ]:
nrows, ncols = 1, 2
fig, axs = plt.subplots(nrows, ncols, figsize=(3 * ncols, 2.5 * nrows), sharey=True)

streamplot(
    axs[0],
    *process_gid(10),
    title=r"$\gamma=0.8\gamma_0$",
    n_skip=6,
    zero_line=True,
    **kwargs
)
streamplot(
    axs[1],
    *process_gid(110),
    title=r"$\gamma=1.8\gamma_0$",
    n_skip=6,
    zero_line=True,
    **kwargs
)
axs[1].set_ylabel("")
axs[0].set_xlabel("")
axs[1].set_xlabel("")
axs[0].tick_params(labelbottom=False)
axs[1].tick_params(labelbottom=False)
plt.subplots_adjust(wspace=0.5)
# fig.savefig("paper_plots/varying_cell_tension_streams_A.pdf")

In [ ]:
nrows, ncols = 1, 2
fig, axs = plt.subplots(nrows, ncols, figsize=(3 * ncols, 2.5 * nrows), sharey=True)

streamplot(axs[0], *process_gid(145), n_skip=6, zero_line=True, **kwargs)
bistable_streamplot(axs[1], *process_gid(108), n_skip=6, zero_line=True, **kwargs)
axs[1].set_ylabel("")

plt.subplots_adjust(wspace=0.5)
# fig.savefig("paper_plots/varying_cell_tension_streams_B.pdf")

### Fig 4

In [ ]:
nrows, ncols = 1, 7
fig, axs = plt.subplots(nrows, ncols, figsize=(2.5 * ncols, 3.5 * nrows), sharey=True)
nskip = 7

gids = [132, 10, 133, 10, 136, 26, 137]
titles = [
    r"$\mu_{\beta}=.63\mathrm{min}^{-1}$",
    r"$\mu_{\beta}=.94\mathrm{min}^{-1}$",
    r"$\mu_{\beta}=1.4\mathrm{min}^{-1}$",
    "",
    "",
    "",
    "",
]

# streams
[
    streamplot(
        ax, *process_gid(gid), title=title, n_skip=nskip, zero_line=True, **kwargs
    )
    for ax, gid, title in zip(axs, gids, titles)
]

# heatmaps
# [
#     heatmap(ax, *process_gid(gid), xlabel=True, ylabel=True)
#     for ax, gid, title in zip(axs, gids, titles)
# ]

[axs[k].set_ylabel("") for k in range(1, ncols)]
axs[3].remove()

plt.tight_layout()
fig.savefig("paper_plots/varying_mub_streams.pdf")

In [ ]:
nrows, ncols = 1, 7
fig, axs = plt.subplots(nrows, ncols, figsize=(2.5 * ncols, 3.5 * nrows), sharey=True)
nskip = 7

gids = [112, 10, 114, 10, 113, 26, 115]
titles = [
    r"$\sigma_{\beta}=2.8\mathrm{min}^{-1}$",
    r"$\sigma_{\beta}=4.7\mathrm{min}^{-1}$",
    r"$\sigma_{\beta}=6.6\mathrm{min}^{-1}$",
    "",
    "",
    "",
    "",
]

# streams
[
    streamplot(
        ax, *process_gid(gid), title=title, n_skip=nskip, zero_line=True, **kwargs
    )
    for ax, gid, title in zip(axs, gids, titles)
]

# heatmaps
# [
#     heatmap(ax, *process_gid(gid), xlabel=True, ylabel=True)
#     for ax, gid, title in zip(axs, gids, titles)
# ]

[axs[k].set_ylabel("") for k in range(1, ncols)]
axs[3].remove()

plt.tight_layout()
fig.savefig("paper_plots/varying_sigmab_streams.pdf")

In [ ]:
nrows, ncols = 1, 7
fig, axs = plt.subplots(nrows, ncols, figsize=(2.5 * ncols, 3.5 * nrows), sharey=True)
nskip = 7

gids = [139, 10, 141, 10, 142, 26, 144]
titles = [
    r"$\sigma=3.4\mu \mathrm{m}$",
    r"$\sigma=6.7\mu \mathrm{m}$",
    r"$\sigma=12\mu \mathrm{m}$",
    "",
    "",
    "",
    "",
]

# streams
for ax, gid, title in zip(axs, gids, titles):
    if gid == 141 or gid == 142:
        bistable_streamplot(
            ax, *process_gid(gid), title=title, n_skip=nskip, zero_line=True, **kwargs
        )
        continue

    streamplot(
        ax, *process_gid(gid), title=title, n_skip=nskip, zero_line=True, **kwargs
    )

# heatmaps
# [
#     heatmap(ax, *process_gid(gid), xlabel=True, ylabel=True)
#     for ax, gid, title in zip(axs, gids, titles)
# ]

[axs[k].set_ylabel("") for k in range(1, ncols)]
axs[3].remove()

plt.tight_layout()
fig.savefig("paper_plots/varying_sigma_streams.pdf")

In [ ]:
nrows, ncols = 1, 7
fig, axs = plt.subplots(nrows, ncols, figsize=(2.5 * ncols, 3.5 * nrows), sharey=True)
nskip = 7

gids = [10, 145, 120, 10, 26, 146, 121]
titles = [
    r"$\tau_{f}=2.2\mathrm{s}$",
    r"$\tau_{f}=3.6\mathrm{s}$",
    r"$\tau_{f}=5.1\mathrm{s}$",
    "",
    "",
    "",
    "",
]

# streams
[
    streamplot(ax, *process_gid(gid), title=title, n_skip=nskip, zero_line=True, **kwargs)
    for ax, gid, title in zip(axs, gids, titles)
]

# heatmaps
# [
#     heatmap(ax, *process_gid(gid), xlabel=True, ylabel=True)
#     for ax, gid, _ in zip(axs, gids, titles)
# ]

[axs[k].set_ylabel("") for k in range(1, ncols)]
# [axs[k].tick_params(labelleft=False) for k in range(1, ncols - 2)]
axs[3].remove()

plt.tight_layout()
fig.savefig("paper_plots/varying_tauf_streams.pdf")